In [1]:
%set_env CUDA_VISIBLE_DEVICES=1
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

print(tf.__version__)
print(keras.__version__)
print(tf.config.list_physical_devices("GPU"))


env: CUDA_VISIBLE_DEVICES=1


2025-05-27 12:21:48.617648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748366508.643885   14723 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748366508.651717   14723 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748366508.671192   14723 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748366508.671211   14723 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748366508.671213   14723 computation_placer.cc:177] computation placer alr

2.19.0
3.9.2
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# load and prepare data for training
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0


## Dense Feed Forward Neural Networks (DFNNs)  also known as multi-layer perceptrons(MLPs)
* well suited for inferencing from tabular data

### Construct DFNN model using Sequential API  
* simple interface
* limited functionality

In [3]:
# Sequential API (Very convenient, not very flexible)
modelDFNN1 = keras.Sequential(
    [
        keras.Input(shape=((28 * 28),), name="input_layer"),
        layers.Dense(512, activation="relu", name="dense_layer1"),
        layers.Dense(256, activation="relu", name="dense_layer2"),
        layers.Dense(
            10, activation="softmax", name="output_layer"
        ),  # added softmax here because crossentropy has fromlogits=False
    ]
)

# modelDFNN1 = keras.Sequential()
# modelDFNN1.add(keras.Input(shape=(784,)))
# modelDFNN1.add(layers.Dense(512, activation="relu"))
# modelDFNN1.add(layers.Dense(256, activation="relu", name="my_layer"))
# modelDFNN1.add(layers.Dense(10))


I0000 00:00:1748366513.447799   14723 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15506 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:82:00.0, compute capability: 6.0


In [4]:
# ToDo #1: List what objects the modelDFNN1 is comprised of (built from)
modelDFNN1.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_layer1 (Dense)            │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_layer2 (Dense)            │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 535,818 (2.04 MB)

 Trainable params: 535,818 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
modelDFNN1.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)


In [6]:
modelDFNN1.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
modelDFNN1.evaluate(x_test, y_test, batch_size=32, verbose=2)


Epoch 1/5


I0000 00:00:1748366515.935612   15077 service.cc:152] XLA service 0x2aae7c001b30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748366515.935646   15077 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-05-27 12:21:55.964837: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748366516.105378   15077 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1748366516.760001   15077 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 - 5s - 3ms/step - accuracy: 0.9433 - loss: 0.1869
Epoch 2/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9752 - loss: 0.0798
Epoch 3/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9821 - loss: 0.0554
Epoch 4/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9870 - loss: 0.0407
Epoch 5/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9892 - loss: 0.0340
313/313 - 1s - 4ms/step - accuracy: 0.9795 - loss: 0.0723


[0.07228989899158478, 0.9794999957084656]

In [7]:
# ToDo #2: What train and test accuracy do you obtain when running for 5 epochs?

print("Training Accuracy: ", modelDFNN1.history.history["accuracy"][-1])
print("Training Loss: ", modelDFNN1.history.history["loss"][-1])

print(
    "Test Accuracy: ", modelDFNN1.evaluate(x_test, y_test, batch_size=32, verbose=0)[1]
)


Training Accuracy:  0.9891666769981384
Training Loss:  0.034011874347925186
Test Accuracy:  0.9794999957084656


In [8]:
# ToDo #3: Now write code (add code cells to this notebook) to train the model for 20 additional epochs. What train and test accuracy do you obtain?

modelDFNN1.fit(x_train, y_train, batch_size=32, epochs=20, verbose=2)


Epoch 1/20
1875/1875 - 3s - 2ms/step - accuracy: 0.9907 - loss: 0.0277
Epoch 2/20
1875/1875 - 3s - 2ms/step - accuracy: 0.9931 - loss: 0.0213
Epoch 3/20
1875/1875 - 3s - 2ms/step - accuracy: 0.9928 - loss: 0.0223
Epoch 4/20
1875/1875 - 3s - 2ms/step - accuracy: 0.9947 - loss: 0.0167
Epoch 5/20
1875/1875 - 3s - 2ms/step - accuracy: 0.9943 - loss: 0.0186
Epoch 6/20
1875/1875 - 3s - 2ms/step - accuracy: 0.9954 - loss: 0.0153
Epoch 7/20
1875/1875 - 3s - 2ms/step - accuracy: 0.9949 - loss: 0.0167
Epoch 8/20
1875/1875 - 3s - 2ms/step - accuracy: 0.9955 - loss: 0.0141
Epoch 9/20
1875/1875 - 3s - 2ms/step - accuracy: 0.9966 - loss: 0.0113
Epoch 10/20
1875/1875 - 3s - 2ms/step - accuracy: 0.9960 - loss: 0.0141
Epoch 11/20
1875/1875 - 3s - 2ms/step - accuracy: 0.9963 - loss: 0.0120
Epoch 12/20
1875/1875 - 3s - 2ms/step - accuracy: 0.9973 - loss: 0.0097
Epoch 13/20
1875/1875 - 3s - 2ms/step - accuracy: 0.9962 - loss: 0.0134
Epoch 14/20
1875/1875 - 3s - 2ms/step - accuracy: 0.9973 - loss: 0.0103
E

In [9]:
print(
    "Training Accuracy after 20 additional epochs: ",
    modelDFNN1.history.history["accuracy"][-1],
)
print(
    "Training Loss after 20 additional epochs: ", modelDFNN1.history.history["loss"][-1]
)
print(
    "Test Accuracy after 20 additional epochs: ",
    modelDFNN1.evaluate(x_test, y_test, batch_size=32, verbose=0)[1],
)


Training Accuracy after 20 additional epochs:  0.9973833560943604
Training Loss after 20 additional epochs:  0.010893275961279869
Test Accuracy after 20 additional epochs:  0.9811000227928162


### Construct DFNN model using Functional API  
* slightly more complex interface
* additional functionality

In [10]:
# ToDo #4: Now practice using the OO functional API of keras by constructing your own functional model, modelDFNN2.
# To simplify the task, create this model so that it uses the same architecture as the sequential model.
# For the final Dense layer use activation="softmax"


In [11]:
# Functional API (A bit more flexible)
inputs = keras.Input(shape=(784,))
x = layers.Dense(512, activation="relu", name="first_layer")(inputs)
x = layers.Dense(256, activation="relu", name="second_layer")(x)
outputs = layers.Dense(10, activation="softmax")(x)
modelDFNN2 = keras.Model(inputs=inputs, outputs=outputs)

modelDFNN2.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ first_layer (Dense)             │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ second_layer (Dense)            │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 535,818 (2.04 MB)

 Trainable params: 535,818 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Now to gain experience invoking methods on objects you have created,
# write code to compile(), fit() your new model on the training data.
# Then write code to evaluate your new model on the test data.


In [13]:
modelDFNN2.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

modelDFNN2.fit(x_train, y_train, batch_size=32, epochs=25, verbose=2)


Epoch 1/25
1875/1875 - 4s - 2ms/step - accuracy: 0.9438 - loss: 0.1867
Epoch 2/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9749 - loss: 0.0801
Epoch 3/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9824 - loss: 0.0555
Epoch 4/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9865 - loss: 0.0426
Epoch 5/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9894 - loss: 0.0322
Epoch 6/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9910 - loss: 0.0279
Epoch 7/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9923 - loss: 0.0233
Epoch 8/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9937 - loss: 0.0203
Epoch 9/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9941 - loss: 0.0190
Epoch 10/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9942 - loss: 0.0177
Epoch 11/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9949 - loss: 0.0163
Epoch 12/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9955 - loss: 0.0152
Epoch 13/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9952 - loss: 0.0160
Epoch 14/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9960 - loss: 0.0139
E

In [14]:
modelDFNN2.evaluate(x_test, y_test, batch_size=32, verbose=2)


313/313 - 1s - 4ms/step - accuracy: 0.9819 - loss: 0.1509


[0.15094128251075745, 0.9818999767303467]

In [15]:
# ToDo #5. Write down what train and test accuracy do you observe? How does it compare to the sequential model?
print(
    "Training Accuracy: ",
    modelDFNN2.history.history["accuracy"][-1],
)
print("Training Loss: ", modelDFNN2.history.history["loss"][-1])
print(
    "Test Accuracy: ",
    modelDFNN2.evaluate(x_test, y_test, batch_size=32, verbose=0)[1],
)


Training Accuracy:  0.9976333379745483
Training Loss:  0.010383560322225094
Test Accuracy:  0.9818999767303467


In [16]:
# For fun: explore the architecture space by adding or removing layers from your model, retrain from scratch.
# Learn how that impacts model performance. State of the art accuracy is around 99.97% .
# You do not need to achieve that (nor is that expected here) but to give you an idea.

# Define new model with more layers, smaller first layer
inputs = keras.Input(shape=(784,))
x = layers.Dense(256, activation="relu", name="first_dense")(inputs)
x = layers.Dense(256, activation="relu", name="second_dense")(x)
x = layers.Dense(256, activation="relu", name="third_dense")(x)
outputs = layers.Dense(10, activation="softmax", name="output")(x)

modelDFNN3 = keras.Model(inputs=inputs, outputs=outputs)
modelDFNN3.summary()
modelDFNN3.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)
modelDFNN3.fit(x_train, y_train, batch_size=32, epochs=25, verbose=2)
modelDFNN3.evaluate(x_test, y_test, batch_size=32, verbose=2)
print(
    "Training Accuracy: ",
    modelDFNN3.history.history["accuracy"][-1],
)
print("Training Loss: ", modelDFNN3.history.history["loss"][-1])
print(
    "Test Accuracy: ",
    modelDFNN3.evaluate(x_test, y_test, batch_size=32, verbose=0)[1],
)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ first_dense (Dense)             │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ second_dense (Dense)            │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ third_dense (Dense)             │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 335,114 (1.28 MB)

 Trainable params: 335,114 (1.28 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
1875/1875 - 5s - 3ms/step - accuracy: 0.9379 - loss: 0.2022
Epoch 2/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9715 - loss: 0.0917
Epoch 3/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9796 - loss: 0.0668
Epoch 4/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9842 - loss: 0.0512
Epoch 5/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9862 - loss: 0.0440
Epoch 6/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9885 - loss: 0.0356
Epoch 7/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9910 - loss: 0.0285
Epoch 8/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9916 - loss: 0.0281
Epoch 9/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9922 - loss: 0.0248
Epoch 10/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9930 - loss: 0.0216
Epoch 11/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9945 - loss: 0.0187
Epoch 12/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9941 - loss: 0.0185
Epoch 13/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9940 - loss: 0.0212
Epoch 14/25
1875/1875 - 3s - 2ms/step - accuracy: 0.9953 - loss: 0.0169
E